## Ustilago maydis mabo code

Manuel Boßmann 13.07.2022 <br/>
file name: iUma22_mabo_feed_plot.ipynb <br/>
This code is for plotting the objective function over a feed from 0% to 100% acetate. The other feed component is glucose.

import alle necessary modules

In [1]:
from os.path import join
from cobra.io import read_sbml_model

Run model iUma22.xml

In [2]:
#Choose either "1.0" iUma22.xml model or "1.1" iUma22v1.1.xml model version

ModelFile = join('..','model','iUma22v1.1.xml')
model=read_sbml_model(ModelFile)

## Co-feeding

choose 1 objective function

In [3]:
#model.objective = "BIOMASS_REACTION"
model.objective = "EX_itacon_e"

Run model from 0 to 100% acetate and output data to copy to excel

In [4]:
total_c_flux = 13.2
# model.reactions.EX_glc__D_e.upper_bound = 0
# model.reactions.EX_glc__D_e.lower_bound = -100
# model.reactions.EX_ac_e.lower_bound = -100
# model.reactions.EX_ac_e.upper_bound = 0
for x in range(0,93,1):
    ac = round(x*total_c_flux/(2*100), 5)
    glc = round(1/6*(total_c_flux-2*ac), 4)

    #either set the feed via medium (#1) or set bounds directly (#2). Sometimes not all of the feed in the medium gets "consumed"
    #by setting upper and lower bound, the model is forced to use all feed
    #however, if it works setting by medium is preferred

    #1
    medium = model.medium
    medium['EX_ac_e']=ac
    medium['EX_glc__D_e']=glc
    model.medium = medium

    #2
    # model.reactions.EX_glc__D_e.upper_bound = -glc
    # model.reactions.EX_glc__D_e.lower_bound = -glc
    # model.reactions.EX_ac_e.lower_bound = -ac
    # model.reactions.EX_ac_e.upper_bound = -ac

    solution = model.optimize()
    
    #theoretically model.summary() should not change to code. However, if the problem has multiple solution
    #the results might change on usage of model.summary()
    model.summary()
    
    
    #for graph ita_cofeed
    print(x, round(solution.fluxes.EX_itacon_e,3), round(-2*solution.fluxes.EX_ac_e/(-2*solution.fluxes.EX_ac_e-6*solution.fluxes.EX_glc__D_e)*100,3), sep="\t")

    #for graph growth rate
    #print(x, round(solution.fluxes.BIOMASS_REACTION,4), sep="\t")

    #for graph C-flux stacked bar chart
    #print(x, round(-solution.fluxes.EX_glc__D_e*6,3), round(-solution.fluxes.EX_ac_e*2,3), sep="\t")

0	2.509	-0.0
1	2.512	1.0
2	2.515	2.0
3	2.518	3.0
4	2.521	4.0
5	2.524	5.0
6	2.527	6.0
7	2.53	7.0
8	2.532	8.0
9	2.535	9.0
10	2.538	10.0
11	2.541	11.0
12	2.544	12.0
13	2.547	13.0
14	2.55	14.0
15	2.553	15.0
16	2.556	16.0
17	2.559	17.0
18	2.562	18.0
19	2.565	19.0
20	2.568	20.0
21	2.571	21.0
22	2.574	22.0
23	2.577	23.0
24	2.58	24.0
25	2.583	25.0
26	2.586	26.0
27	2.589	27.0
28	2.592	28.0
29	2.595	29.0
30	2.598	30.0
31	2.601	31.0
32	2.604	32.0
33	2.607	33.0
34	2.61	34.0
35	2.613	35.0
36	2.616	36.0
37	2.619	37.0
38	2.622	38.0
39	2.624	39.0
40	2.625	40.0
41	2.625	41.0
42	2.624	42.0
43	2.618	43.0
44	2.611	44.0
45	2.603	45.0
46	2.596	46.0
47	2.589	47.0
48	2.581	48.0
49	2.574	49.0
50	2.567	50.0
51	2.559	51.0
52	2.552	52.0
53	2.545	53.0
54	2.537	54.0
55	2.53	55.0
56	2.523	56.0
57	2.515	57.0
58	2.508	58.0
59	2.501	59.0
60	2.493	60.0
61	2.486	61.0
62	2.479	62.0
63	2.471	63.0
64	2.464	64.0
65	2.42	64.407
66	2.332	64.085
67	2.244	63.736
68	2.156	63.359
69	2.068	62.948
70	1.98	62.5
71	1.892	62.009
72	1.8

This is just for testing stuff

In [5]:
def test_cofeed(ac, glc):
    medium = model.medium
    medium['EX_ac_e']=ac
    medium['EX_glc__D_e']=glc
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes.BIOMASS_REACTION)
    return solution

In [6]:
test_cofeed(0,1)
# test_cofeed(0.25,1)
# test_cofeed(0.5,1)
test_cofeed(1,1)

0.0
0.0


,fluxes,reduced_costs
ANTPPT,0.0,-4.099548
ATNS,0.0,0.000000
TRANS-RXNTFS-55-CPD-125752,0.0,0.000000
TRANS-RXNTFS-55-DTDP-D-GLUCOSE3,0.0,0.000000
TRANS-RXNTFS-55-ADP-D-GLUCOSE4,0.0,0.000000
...,...,...
3MOPDECARB,0.0,0.000000
EX_2obut_e,0.0,-1.339367
EX_for_e,0.0,-0.389140
PseudoMan6P2Mnl,0.0,-0.072398
